### **Hani Soni**

### **202318044**

Task B: The API call process and dataset creation.

Required liabraries.

In [1]:
import json
import logging
from datetime import datetime, timedelta
import pandas as pd
import requests

Define a class `Event` representing an event object with 11 fields, providing default values for each field and generating a unique object ID based on the current date and time.

In [4]:
def create_unique_object_id():
    now = datetime.now()
    formatted_value = now.strftime("%y%m%d%f")
    return formatted_value


class Event(dict):

    """
        Event class used to represent an event object has 32 fields.
    """

    def __init__(self, *args, **kwargs):

        id = create_unique_object_id()
        default_data = {
            "id": id,
            "title": "",
            "sdate": "",
            "stime": "",
            "address": "",
            "latitude": "",
            "longitude": "",
            "place_name": "",
            "event category": "",
            "event_url": "",
            "original_img_url": ""
        }

        super().__init__(*args, **default_data)

        del default_data  # removing as it's of no use now


Define a function `fetch_events_from_ticketmaster` that queries the Ticketmaster API for events related to the keyword "religious" in a specified city and time range, processes the API response, and creates a list of `Event` objects with relevant information such as title, date, time, location, and URL.

In [5]:
def fetch_events_from_ticketmaster(city="Miami", days=30):
    start_time = datetime.now()
    end_time = start_time + timedelta(days=days)
    start_time = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    print(start_time)

    logging.info(f"Start Time: {start_time}, End Time: {end_time}")
    events = []
    url = f"https://app.ticketmaster.com/discovery/v2/events.json?apikey=iTkEPL9j8mfVUGeTgYSADIphb7pXDMOG&keyword=religious&size=200&page=0&startDateTime={start_time}&endDateTime={end_time}"

    print(url)
    try:
        results = []
        response = requests.get(url=url)  # API calling

        response = response.json()
        response = response["_embedded"]["events"]
        logging.info(f"Number of events: {len(response)}")
        for i, record in enumerate(response):
            print(i)
            try:
                if record.get("images") != []:
                    url = record["images"][4].get("url")
                an_event = Event()
                an_event['title'] = record.get("name")
                # data:startdate
                an_event['sdate'] = record["dates"]["start"]["localDate"]
                # data:starttime
                an_event['stime'] = record["dates"]["start"]["localTime"]
                # data:event_address
                an_event['address'] = f'{record["_embedded"]["venues"][0]["name"]} {record["_embedded"]["venues"][0]["postalCode"]}'
                # data:latitude
                an_event['latitude'] = record["_embedded"]["venues"][0]["location"]["latitude"]
                an_event['longitude'] = record["_embedded"]["venues"][0]["location"]["longitude"]
                # data:place_name
                an_event['place_name'] = record["_embedded"]["venues"][0]["address"]["line1"]
                # data:categories
                an_event['event category'] = record["classifications"][0]["segment"]["name"]
                an_event['event_url'] = record["url"]  # data:url
                an_event['original_img_url'] = url  # data:original_img_name
                results.append(an_event)
                print(
                    f"Event {i} processed successfully. {an_event['event_url']}")
                logging.info(
                    f"Event {i} processed successfully. {an_event['event_url']}")
            except KeyError:
                logging.exception(
                    f"KeyError: occurred while processing Event {i}. url = {an_event['event_url']}")
    except requests.exceptions.RequestException:
        logging.exception(f"Request Exception occurred.")
    except ValueError:
        logging.exception(
            f"ValueError: occurred while processing the response JSON.")
    except Exception:
        logging.exception(f"An error occurred.")

    return results

This code fetches events within the next 730 days, creates a DataFrame from the obtained data, saves it to an Excel file ("ticketmaster.xlsx").

In [ ]:
events = fetch_events_from_ticketmaster(days=730)

df = pd.DataFrame(events)
df.to_excel("ticketmaster.xlsx")  # dataset generation